In [0]:
# Databricks notebook source
# MAGIC %pip install databricks-sdk==0.30.0 
# MAGIC dbutils.library.restartPython()


# COMMAND
# Import necessary libraries
import importlib.metadata
print(importlib.metadata.version("databricks-sdk"))


import time
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import sql
from databricks.sdk.service.jobs import CronSchedule, SqlTask, PauseStatus, SqlTaskAlert, Task, SqlTaskSubscription
from databricks.sdk.service.iam import AccessControlRequest, PermissionLevel


# --- CONFIGURATION ---
TABLE_NAME = "jpmc_group_catalog.mlops.wine_predictions_v2_dabs"
QUERY_NAME = "wine Inference Table Row Count Monitor"
ALERT_NAME = "wine Inference Table Row Count Alert"
THRESHOLD = 1000
SCHEDULE_INTERVAL_MINUTES = 15
NOTIFICATION_EMAIL = "mlops_databricks_notifications@jpmchase.com" 
WAREHOUSE_ID= "682a84ec8bdc04dd"

w = WorkspaceClient()

# 1. CREATE QUERY
print("\nCreating query...") 
query = w.queries.create(
   query=sql.CreateQueryRequestQuery( 
      display_name=QUERY_NAME,
      warehouse_id=WAREHOUSE_ID,
      description="Monitors row count for wine inference table", 
      query_text=f"SELECT COUNT(*) AS row_count FROM {TABLE_NAME};",
      run_as_mode=sql.RunAsMode.VIEWER,
   )
)

print(f"Created query: {query.id}")

In [0]:

print(f"✅ Created query: {query.id}")

# --- 2. CREATE ALERT WITH NOTIFICATION ---
print("\nCreating alert with notification...")
alert = w.alerts.create(
    alert=sql.CreateAlertRequestAlert(
        display_name=ALERT_NAME,
        query_id=query.id,
        condition=sql.AlertCondition(
            operand=sql.AlertConditionOperand(
                column=sql.AlertOperandColumn(name="row_count")
            ),
            op=sql.AlertOperator.GREATER_THAN,
            threshold=sql.AlertConditionThreshold(
                value=sql.AlertOperandValue(double_value=float(THRESHOLD))
            ),
        ),
        custom_subject=f"🚨 [{ALERT_NAME}] Row Count Threshold Exceeded",
        custom_body=f"""
DATABRICKS ALERT NOTIFICATION

Alert Name: {{{{ALERT_NAME}}}}
Triggered: {{{{ALERT_TRIGGERED_AT}}}}
Query: {{{{QUERY_NAME}}}}

THRESHOLD BREACH
Current Value: {{{{QUERY_RESULT_VALUE}}}} rows
Threshold: > {THRESHOLD} rows
Status: EXCEEDED

TABLE: {TABLE_NAME}
""",
        seconds_to_retrigger=36000,
    )
)

print(f"✅ Created alert: {alert.id}")


In [0]:

job_name = "Wine Inference Alert Job"
cron_expression = "0 0 * * * ?"
timezone = "UTC"


sql_task = SqlTask(
    alert=SqlTaskAlert(
        alert_id=alert.id,
        subscriptions=[
            SqlTaskSubscription(destination_id="c76b8ca9-b04a-4b7e-9f49-0a83efd31fbb")
        ]
    ),
    warehouse_id=WAREHOUSE_ID
)

job = w.jobs.create(
    name=job_name,
    schedule=CronSchedule(
        quartz_cron_expression=cron_expression,
        timezone_id=timezone,
        pause_status=PauseStatus.UNPAUSED
    ),
    tasks=[
        Task(
            task_key="alert_task",
            sql_task=sql_task
        )
    ]
    #email_notifications=JobEmailNotifications(
    #   on_failure=["m14hr_databricks_notifications@jpmchase.com"],
    #   on_success=["m14hr_databricks_notifications@jpmchase.com"],
    #   no_alert_for_skipped_runs=False
    #)
)

print(f"Job created successfully!")
print(f"Created Job with ID: {job.job_id} to schedule the alert.")
